In [16]:
#Importamos librerias
import pandas as pd
import re

In [ ]:
#Importamos los datos de los restaurantes de google
google = pd.read_json('metadata-sitios/google_sitios.json')
google.head()

In [ ]:
#Examinamos un poco más
google.info()

Como primera medida vamos a normalizar los campos. Lo que buscamos es tener una tabla que contenga las siguientes columnas:

* Id_Restaurant
* Nombre
* Ciudad
* Estado
* Código postal
* Latitud
* Longitud
* Tipo (category)
* Atributos (MISC)

La información sobre la ciudad, estado y código postal se encuentra bajo el campo address del dataframe de google. Vamos a separarlo.

In [ ]:
#Quitamos los valores que no tengan el valor de address
restaurants_google = google.dropna(subset='address')
restaurants_google.reset_index(inplace=True)
restaurants_google.info()

In [ ]:
#Separamos y juntamos los datos
listado_fallas = []
localizaciones = []
for i in range(restaurants_google['address'].values.size):
    try:
        int(restaurants_google['address'].values[i][-1])
        if len(restaurants_google['address'].values[i].split(',')[-1].strip()) == 8:
            localizaciones.append([restaurants_google['gmap_id'].values[i],                                     #Id del restaurant
                                restaurants_google['address'].values[i].split(',')[-2].strip(),                 #Ciudad
                                restaurants_google['address'].values[i].split(',')[-1].strip().split()[0],      #Estado
                                restaurants_google['address'].values[i].split(',')[-1].strip().split()[1]])     #Código postal
        else:
            listado_fallas.append(i)                                    #Anoto cuantas direcciones no me dan los datos necesarios
    except:
        listado_fallas.append(i)
len(listado_fallas)

In [ ]:
#Armamos dataframe con estos datos
localizacion = pd.DataFrame(columns=['gmap_id','Ciudad','Estado','Cod_postal'],data=localizaciones)
localizacion.head()

In [ ]:
#Realizamos una comprobación. Vemos si algún valor falló
fallados = []
for i in range(localizacion.shape[0]):
    try:
        int(localizacion['Cod_postal'].iloc[i])                 #Si esto funciona quiere decir que no entraron caracteres extraños
    except:                                                     #bajo cod_postal
        print(i,': ',localizacion['Cod_postal'].iloc[i])
        fallados.append(localizacion['Cod_postal'].iloc[i])

In [ ]:
#No fallo. 12590 registros no cuentan con los valores que buscamos. Los observamos a ver si se puede hacer algo.
restaurants_google[restaurants_google.index.isin(listado_fallas)]

In [ ]:
#Observamos el campo address más de cerca
list(restaurants_google[restaurants_google.index.isin(listado_fallas)]['address'].values)

Vemos que hay muchos que tienen "United States" al final. Esto trae problemas en el algoritmo planteado. Modificando ligeramente podemos incorporar estos datos.

In [ ]:
restaurants_google_fallas = restaurants_google[restaurants_google.index.isin(listado_fallas)]
restaurants_google_fallas.reset_index(inplace=True)
restaurants_google_fallas.info()

In [ ]:
listado_fallas = []
for i in range(restaurants_google_fallas['address'].values.size):
    try:
        lista_address = restaurants_google_fallas['address'].values[i].split(',')[0:4]      #Cortamos el "United States" de los -->
        int(lista_address[-1][-1])                                                          #registros que los tienen.
        if len(lista_address[-1].strip()) == 8:
            localizaciones.append([restaurants_google_fallas['gmap_id'].values[i],          #Id del restaurant
                                   lista_address[-2].strip(),                               #Ciudad
                                   lista_address[-1].strip().split()[0],                    #Estado
                                   lista_address[-1].strip().split()[1]])                   #Codigo postal
        else:
            listado_fallas.append(i)                                    
    except:
        listado_fallas.append(i)
len(listado_fallas)

In [ ]:
#Armamos dataframe con estos datos
localizacion = pd.DataFrame(columns=['gmap_id','Ciudad','Estado','Cod_postal'],data=localizaciones)
localizacion.head()

In [ ]:
#Realizamos una comprobación. Vemos si algún valor falló
fallados = []
for i in range(localizacion.shape[0]):
    try:
        int(localizacion['Cod_postal'].iloc[i])                 #Si esto funciona quiere decir que no entraron caracteres extraños
    except:                                                     #bajo cod_postal
        print(i,': ',localizacion['Cod_postal'].iloc[i])
        fallados.append(localizacion['Cod_postal'].iloc[i])

In [ ]:
#Se redujeron a 1008 casos nomas. Volvemos a observar.
restaurants_google_fallas[restaurants_google_fallas.index.isin(listado_fallas)]